In [6]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,dense_rank,when,sum
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [2]:
Projects = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Projects") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Candidates = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Candidates") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [16]:
Projects\
    .join(Candidates,Projects.skill == Candidates.skill,"inner")\
    .select("candidate_id",Candidates.skill,"proficiency","project_id","importance")\
    .withColumn("score",when(col("proficiency")>col("importance"),10)\
    .when(col("proficiency")==col("importance"),0).otherwise(-5))\
    .groupby(col("project_id"),col("candidate_id"))\
    .agg((sum("score")+100).alias("score"))\
    .withColumn("drnk",dense_rank().over(Window.partitionBy("project_id").orderBy(col("score").desc(),"candidate_id")))\
    .filter("drnk==1")\
    .drop("drnk")\
    .show()

+----------+------------+-----+
|project_id|candidate_id|score|
+----------+------------+-----+
|       501|         101|  105|
|       502|         102|  130|
+----------+------------+-----+

